Service Mesh - Istio
--------------------

![](https://istio.io/latest/docs/ops/deployment/architecture/arch.svg)

Quelle: istio.io

- - -

Cloud-Plattformen bieten den Unternehmen, die sie verwenden, zahlreiche Vorteile.

Dazu müssen Entwickler das Architekturmuster Microservices verwenden, was dazu führt das schnell Unterschiedliche Versionen von Microservices parallel betrieben werden müssen.

Mit den vielfältigen Funktionen von [Istio](http://istio.io) kann man eine verteilte Microservice-Architektur erfolgreich und effizient ausführen und auf einheitliche Weise Microservices absichern, verbinden und überwachen.

Es ist ein vollständiger Open-Source-Service-Mesh, das auf vorhandenen verteilten Anwendungen aufbaut.

***
### Was ist ein Service-Mesh?

Der Begriff Service-Mesh beschreibt das Netzwerk von Microservices und die Wechselwirkungen zwischen ihnen. 

Je grösser und komplexer ein Service-Mesh wird, desto schwieriger ist es, es zu verstehen und zu verwalten. 

Zu den Anforderungen zählen Erkennung, Lastenausgleich, Fehlerbehebung, Metriken und Überwachung. 

Ein Service-Mesh hat häufig komplexere betriebliche Anforderungen, wie A/B-Tests (Bewertung zweier Varianten), Canary-Rollouts, Beschränkungen, Zugangskontrolle und End-to-End-Authentifizierung.

Weitere Informationen: [Istio Homepage](https://istio.io/docs/concepts/what-is-istio/)

***
### Installation

Istio ist in der [duk-demo](https://github.com/mc-b/duk-demo) Umgebung bereits installiert. Ansonsten siehe Projekt [duk](https://github.com/mc-b/duk/tree/master/istio).

***
### Bookinfo Beispiel

Die Beispielanwendung zeigt Informationen zu einem Buch an, ähnlich einem einzelnen Katalogeintrag eines Online-Buchladens. Auf der Seite werden eine Beschreibung des Buches, Buchdetails (ISBN, Seitenzahl usw.) und einige Buchbesprechungen angezeigt.

Für Details siehe [Bookinfo Application](https://istio.io/docs/examples/bookinfo/) und den [Sourcecode auf Github](https://github.com/istio/istio/tree/master/samples/bookinfo). 

Die Beispielanwendung ist Bestandteil der Istio Sourcen. Deshalb müssen wir diese zuerst downloaden, dann erstellen wir einen eigenen Namespace wo istio aktiviert ist und darin die Bookinfo Beispielanwendung.


In [ ]:
! curl -s -L https://istio.io/downloadIstio | ISTIO_VERSION=1.6.8 sh -
! ls -l istio-1.6.8/samples/bookinfo

In [ ]:
! kubectl create namespace bookinfo
! kubectl label namespace bookinfo istio-injection=enabled
! kubectl apply -f istio-1.6.8/samples/bookinfo/platform/kube/bookinfo.yaml -n bookinfo
! kubectl apply -f istio-1.6.8/samples/bookinfo/networking/bookinfo-gateway.yaml -n bookinfo
! kubectl apply -f istio-1.6.8/samples/bookinfo/networking/destination-rule-all.yaml -n bookinfo

Neben Pods, Services etc. wurden auch weitere Ressourcen erstellt wie:
* [Gateway](https://istio.io/docs/concepts/traffic-management/#gateways) - für den Zugriff von ausserhalb des Clusters
* [Virtual Service](https://istio.io/docs/concepts/traffic-management/#virtual-services) - mit den URL Paths 
* [Routing Rule](https://istio.io/docs/concepts/traffic-management/#routing-rules) - mit den Regeln was wo weitergeleitet wird.


In [ ]:
! kubectl get all,gw,vs,dr -n bookinfo

In [ ]:
! kubectl get gw,vs,dr -n bookinfo -o yaml

Schauen wir uns aber zuerst die Applikation an. 

![](https://istio.io/latest/docs/examples/bookinfo/noistio.svg)

Quelle: istio.io

- - - 

Die Bookinfo-Anwendung ist in vier separate Microservices unterteilt:

* productpage: Der productpage Microservice ruft die Microservices details und reviews auf, um die Seite zu füllen.
* details: Der details Microservice enthält Buchinformationen.
* reviews: Der reviews Microservice enthält Buchbesprechungen. Es ruft auch den ratings Microservice auf.
* ratings: Der ratings Microservice enthält Buchranginformationen, die einer Buchbesprechung beigefügt sind.

Es gibt 3 Versionen des reviews Microservice:
* Version v1 ruft den ratings Microservice nicht auf.
* Version v2 ruft den ratings Microservice auf und zeigt jede Bewertung als 1 bis 5 schwarze Sterne an.
* Version v3 ruft den ratings Microservice auf und zeigt jede Bewertung als 1 bis 5 rote Sterne an.

Diese Anwendung sind in mehreren Programmiersprache implementiert.

Die Microservices enthalten keine Abhängigkeiten und wissen auch nichts von Istio. Sie wurden auch nicht speziell Angepasst um mit Istio zu funktionieren.

Das User Interface kann wie folgt angewählt werden:

In [ ]:
! echo "$(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl --namespace istio-system get service istio-ingressgateway -o jsonpath='{ .spec.ports[1].nodePort }')/g")/productpage"

Wählen wir die Seite mehrmals an, wird abwechselnd einer der drei Version vom Microservice `reviews` aufgerufen.

Um gezielt:

* nur eine Version aufzurufen, Version 1 oder 3
* Analog eines Canary Rollout 50/50 Prozent Version 2 und 3 aufzurufen

stehen vorbereitete YAML Dateien zu Verfügung. Führen Sie diese jeweils einzeln aus und Testen im UI den Effekt. Dabei kann 1 - 2 Minuten dauern, bis die Regeln aktiv werden.

In [ ]:
! kubectl apply -f istio-1.6.8/samples/bookinfo/networking/virtual-service-all-v1.yaml -n bookinfo

In [ ]:
! kubectl apply -f istio-1.6.8/samples/bookinfo/networking/virtual-service-reviews-v3.yaml -n bookinfo

In [ ]:
! kubectl apply -f istio-1.6.8/samples/bookinfo/networking/virtual-service-reviews-v2-v3.yaml -n bookinfo

*** 
### Tools

#### Kiali — Observability

[Kiali](https://www.kiali.io/) ist eine Web UI für Istio. Es hilft Ihnen, die Struktur Ihres Servicenetzes und deren Topologie zu verstehen.
  
#### Jaeger — Tracing

Die Ablaufinformationen für Kiali werden durch das verteiltes Tracing-System [Jaeger](https://www.jaegertracing.io/) bereitgestellt.

Beide Tools sind jedoch nur auf dem Cluster zugänglich. Deshalb müssen wir diese zuerst freischalten bzw. gegen NodePort öffnen:

In [ ]:
! kubectl get service -n istio-system -l app=kiali -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! kubectl get service jaeger-query -n istio-system -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -

Sind Sie freigeschaltet, können wir Sie wie andere Services via Cluster-IP und Port erreichen:

In [ ]:
! echo "Kiali  UI: " $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service -n istio-system -l app=kiali -o yaml -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')/g")
! echo "Jeager UI: " $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service jaeger-query -n istio-system -o=jsonpath='{ .spec.ports[0].nodePort }')/g")  

Aufräumen

In [ ]:
! kubectl delete namespace bookinfo

***

Weitere Beispiele siehe Projekt [duk](https://github.com/mc-b/duk/tree/master/istio).
